In [51]:
import pandas as pd

df = pd.concat(pd.read_excel("Folds5x2_pp.xlsx", sheet_name=None), ignore_index=True)

train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

def normalize(df):
    for key in df.keys():
        df[key] = df[key] / df[key].max()
    return df.to_numpy()


train_features = normalize(train_dataset.drop(["PE"], axis=1))
test_features = normalize(test_dataset.drop(["PE"], axis=1))
train_labels = train_dataset["PE"].values.reshape(1, -1)[0]
test_labels = test_dataset["PE"].values.reshape(1, -1)[0]

In [52]:
import numpy as np

input_size = len(train_features[0])

# DAMN NETWORK
w1 = np.random.rand(input_size, 3)
b1 = np.random.rand(3,)
w2 = np.random.rand(3, 1)
b2 = np.random.rand(1,)

In [53]:
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))

def d_sigmoid(z):
    return sigmoid(z)*(1-sigmoid(z))

def forward(x):
    x = x.dot(w1)
    x = x + b1
    x = sigmoid(x)
    x = x.dot(w2)
    x = x + b2
    return x

def sqrt_mean_squared_error(x, y):
    return np.square(x - y).sum() / len(x)

In [63]:
# TRAINING DATA
# function to create a list containing mini-batches
def create_mini_batches(X, y, batch_size):
    mini_batches = []
    data = np.hstack((X, y))
    np.random.shuffle(data)
    n_minibatches = data.shape[0] // batch_size
    i = 0
  
    for i in range(n_minibatches + 1):
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    if data.shape[0] % batch_size != 0:
        mini_batch = data[i * batch_size:data.shape[0]]
        X_mini = mini_batch[:, :-1]
        Y_mini = mini_batch[:, -1].reshape((-1, 1))
        mini_batches.append((X_mini, Y_mini))
    return mini_batches

# create batches that contains 50 (X,y) each
mini_batches = create_mini_batches(train_features, train_labels.reshape(len(train_labels), 1), 50) 

In [67]:
# THIS ENTIRE PROCESS COUNT AS AN EPOCH

# LEARNING_RATE
λ = 0.01

# GRADIENT TO BE
dJdB1 = [np.zeros(b.shape) for b in w1]
dJdW1 = [np.zeros(w.shape) for w in w1]
dJdB2 = [np.zeros(b.shape) for b in w2]
dJdW2 = [np.zeros(w.shape) for w in w2]

# TODO => USE MINI_BATCHES NOT MINI_BATCH
mini_batch = zip(train_features, train_labels)
for x, y in mini_batch:
#     batch_size = len(x)
    # FORWARD PROPAGATION
    # Z1(3,1) = ((W1(4,3).T(3,4) * X(4,1))(3,1) + b(3,1))
    z1 = w1.T.dot(x) + b1
    # A1(3,1) = A(Z1(3,1))
    a1 = sigmoid(z1)
    
    # USING RESHAPE CAUSE NUMPY (3,1) != (3,)
    # Z2(1,1) = ((W2(3,1).T(1,3) * A1(3,1))(2,1) + b(1,1))
    z2 = (w2.reshape(3,).T.dot(a1) + b2)
    # A2(2,1) = A(Z2(2,1))
    a2 = sigmoid(z2)
    
    # BACKWARD PROPAGATION
    # HERE WE DO MATHS, NO LUCK, HIGHLY SUGGEST https://www.youtube.com/watch?v=tIeHLnjs5U8
    # BY FOLLOWING CHAIN RULE =>
    # dJdW2(3,1) = dZ2/dW2 * dA2/dZ2 * dJ/dA2 with J our cost function
    dJdA2 = (2 * (a2 - y))
    dZ2dW2 = a1
    dA2dZ2 = d_sigmoid(z2)
    dJdW2 = dZ2dW2 * dA2dZ2 * dJdA2
    dZ2dB2 = 1
    dJdB2 = dZ2dB2 * dA2dZ2 * dJdA2
    
    # dJdW1(4,3) = dZ1/dW1 * dA1/dZ1 * dJ/dA1 with J our cost function
    # HERE THE HARD PART IS ABOUT dJ/A1, which have impact on cost function by having impact on Z2 -> A2 -> J
    # THE NOT COOL PART IS ABOUT HAVING THOSE MATRICE DIMENSION RIGHT LOL
    dJdZ1 = d_sigmoid(z1).reshape(3,1) * w2.reshape(3,1) * d_sigmoid(z2).reshape(1,1) * dJdA2.reshape(1,1)
    dJdW1 = x.reshape(4,1).dot(dJdZ1.T)
    dZ1dB1 = 1
    dJdB1 = dJdZ1.T * 1
    
    print(dJdW2.shape)
    print(dJdB2.shape)
    # (3,1)
    # (1,1)
    # (4,3)
    # (3,1)
    print(dJdW1.shape)
    print(dJdB1.shape)
    
    break

# d = len(train_features)
# w1 = [w - λ/d * dw for w, dw in zip(w1, dJdW1)]
# w2 = [w - λ/d * dw for w, dw in zip(w2, dJdW2)]
# b1 = [b - λ/d * dw for b, dw in zip(b1, dJdB1)]
# b2 = [b - λ/d * dw for b, dw in zip(b2, dJdB2)]
# print(w1)

(3,)
(1,)
(4, 3)
(1, 3)


In [56]:
np.array([1,2,3]) * np.array([3,2,1])

array([3, 4, 3])